In [1]:
# nth attempt to generate ODE eqs (Von Neumann RHS) 
# given a hamiltonian. and eventually be able to make the RWA

In [4]:
import numpy as np
from sympy.parsing.sympy_parser import parse_expr
from sympy import MatrixSymbol,MatAdd,MatMul,Identity,I,Matrix,symbols
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.utilities.iterables import flatten
import sympy as sp

In [53]:
#### functions

def comm(A,B):
    """
    return commutator(A, B)[A,B]=A.B-B.A. Assumes symbolic matrix.
    """
    assert A.shape == B.shape, (f"A,B must have the same shape, but"+
                                f"A.shape={A.shape} != B.shape={B.shape}")
    return (MatMul(A,B)-MatMul(B,A)) #.as_mutable

def RHS(H):
    """ return derivatives function with call signature:
            derivs(y0, t) where y0 = [r11, r12, ...] 
            is the unraveled non-redundant density matrix
    """
    
    pass

In [36]:
#### here's how I want to call my RHS function

# define symbolic matrices which can be made numeric later
E1,E2,d,O,t = symbols('E1 E2 d O t') # symbolic variables

# Set the parameters of H; arbitrary for now
O = 1; E2 = 1; E1 = 0; d = 1;

# Declare the atom and interaction Hamiltonians
H_a = Matrix([[E1,0],[0,E2]])
H_field = Matrix([[0,-cc(O)/2],[-O/2,0]])
H = H_a + H_field # the full Hamiltonian

# dervis = RHS(H)

In [83]:
# build a symbolic density matrix
dims = 2
r = MatrixSymbol('r', dims, dims).as_mutable()
for i in range(dims):
    for j in range(dims):
        r[i,j] = symbols(f'r{i}{j}')
        if i > j:
            r[i,j] = np.conj(r[j,i])
print(r)

# calculate [r, H]:
rhs = comm(r, H)
print("RHS eqs:")
for eq in rhs:
    print(eq)
    
# prune off non-redundant elements
pruned_rhs = []
for i in range(dim):
    for j in range(dim):
        if i >= j:
            pruned_rhs.append(rhs[i,j])
    
# wrap rhs in a function 
args = list(rhs.free_symbols)
args.append(symbols('t'))
args.sort(key=lambda x: x.__repr__())


f = lambdify(args, pruned_rhs)
help(f)

# okay for time independent Hamiltonian, but need to work 
# out for when H has explicit t dependence
def derivs(t, y):  
    return f

Matrix([[r00, r01], [conjugate(r01), r11]])
RHS eqs:
-0.5*r01 + 0.5*conjugate(r01)
-0.5*r00 + r01 + 0.5*r11
0.5*r00 - 0.5*r11 - conjugate(r01)
0.5*r01 - 0.5*conjugate(r01)
Help on function _lambdifygenerated:

_lambdifygenerated(r00, r01, r11, t)
    Created with lambdify. Signature:
    
    func(r00, r01, r11, t)
    
    Expression:
    
    [-0.5*r01 + 0.5*conjugate(r01), 0.5*r00 - 0.5*r11 - conjugate(r01), 0.5*r01...
    
    Source code:
    
    def _lambdifygenerated(r00, r01, r11, t):
        return ([-0.5*r01 + 0.5*conjugate(r01), 0.5*r00 - 0.5*r11 - conjugate(r01), 0.5*r01 - 0.5*conjugate(r01)])
    
    
    Imported modules:



In [75]:
# now, try to solve it:

Matrix([
[   0, -0.5],
[-0.5,    1]])